In [1]:
from pydantic import BaseModel, Field
import os, sqlite3, json
from dotenv import load_dotenv
from openai import OpenAI
from langsmith.wrappers import wrap_openai
from langsmith import traceable
from openai import pydantic_function_tool
load_dotenv()

# read in env variables
#api_key = os.getenv("OLLAMA_API_KEY")
#api_base = os.getenv("OLLAMA_API_BASE")
#model = os.getenv("QWEN3_API_MODEL")

api_key = os.getenv("GEMINI_API_KEY")
api_base = os.getenv("GEMINI_API_BASE")
model = os.getenv("GEMINI_API_MODEL")

In [2]:
client = wrap_openai(OpenAI(
 api_key=api_key,
 base_url=api_base,
))

In [3]:
class Album(BaseModel):
    index: int = Field(description="The database index of the album")
    year: int = Field(description="The year the album was released")
    title: str = Field(description="The title of the album")
    artist: str = Field(description="The name of the artist who made the album")
    genre: str = Field(description="The music genre of the album")
    subgenre: str = Field(description="The music subgenre of the album")
    price: float = Field(description="The orice of the album")

class AlbumList(BaseModel):
    albums: list[Album]

In [4]:
class getAlbumByTitle(BaseModel):
    """Tool that queries albums by title and returns an Album object containing the database index, Album Year, Album Title, Artist Name, Album Genre, Album Subgenre  and Album Price"""
    albumTitle: str = Field(description="The title of the album.")

    def run(self):
        try:
            with sqlite3.connect('music.db') as connection:
                cursor = connection.cursor()

                sql_query = "SELECT * FROM music WHERE album = ?"

                cursor.execute(sql_query,[self.albumTitle])
                result = cursor.fetchone()

                if result is None:
                    return "No album found with that title"

                keys = ['index','year','title','artist','genre','subgenre','price']
                vals = list(result)

                result_dict = dict(zip(keys, vals))

                album = Album(**result_dict)

                return album.model_dump_json()


        #print("Database connection closed successfully!")
        except sqlite3.IntegrityError as e:
            print(f"Error: Integrity constraint violated - {e}")

        except sqlite3.OperationalError as e:
            print(f"Error: Operational issue - {e}")

        except sqlite3.Error as e:
            print(f"Error: Generic Sqlite3 error - {e}")        

        except Exception as e:
            print(f"An unexpected error occurred: {e}")


In [28]:
class getAlbumByArtist(BaseModel):
    """Tool that queries albums by Artist and returns a list of  Album objects containing the database index, Album Year, Album Title, Artist Name, Album Genre, Album Subgenre  and Album Price"""
    albumArtist: str = Field(description="The Artist name.")

    def run(self):
        try:
            with sqlite3.connect('music.db') as connection:
                cursor = connection.cursor()

                #sql_query = "SELECT * FROM music WHERE artist = '%s'" % self.albumArtist
                sql_query = "SELECT * FROM music WHERE artist LIKE ?"

                cursor.execute(sql_query,['%' + self.albumArtist + '%'])
                results = cursor.fetchall()
                #print(f"Results from get album by artist : {results}")

                albums = []

                for result in results:
                    keys = ['index','year','title','artist','genre','subgenre','price']
                    vals = list(result)

                    result_dict = dict(zip(keys, vals))

                    album = Album(**result_dict)
                    albums.append(album.model_dump_json())
                    print(f"Albums returned by Artist: {albums}")

                return albums


        #print("Database connection closed successfully!")
        except sqlite3.IntegrityError as e:
            print(f"Error: Integrity constraint violated - {e}")

        except sqlite3.OperationalError as e:
            print(f"Error: Operational issue - {e}")

        except sqlite3.Error as e:
            print(f"Error: Generic Sqlite3 error - {e}")        

        except Exception as e:
            print(f"An unexpected error occurred: {e}")

In [26]:
class getAlbumByYear(BaseModel):
    """Tool that queries albums by Year and returns a list of  Album objects containing the database index, Album Year, Album Title, Artist Name, Album Genre, Album Subgenre  and Album Price"""
    albumYear: str = Field(description="The Year.")

    def run(self):
        try:
            with sqlite3.connect('music.db') as connection:
                cursor = connection.cursor()

                sql_query = "SELECT * FROM music WHERE year = ?"

                cursor.execute(sql_query, [self.albumYear])
                results = cursor.fetchall()

                albums = []

                for result in results:
                    keys = ['index','year','title','artist','genre','subgenre','price']
                    vals = list(result)

                    result_dict = dict(zip(keys, vals))

                    album = Album(**result_dict)
                    albums.append(album.model_dump_json())

                return albums


        #print("Database connection closed successfully!")
        except sqlite3.IntegrityError as e:
            print(f"Error: Integrity constraint violated - {e}")

        except sqlite3.OperationalError as e:
            print(f"Error: Operational issue - {e}")

        except sqlite3.Error as e:
            print(f"Error: Generic Sqlite3 error - {e}")        

        except Exception as e:
            print(f"An unexpected error occurred: {e}")

In [7]:
class getAlbumByGenre(BaseModel):
    """Tool that queries albums by Genre and returns a list of  Album objects containing the database index, Album Year, Album Title, Artist Name, Album Genre, Album Subgenre  and Album Price"""
    albumGenre: str = Field(description="The Album Genre name.")

    def run(self):
        try:
            with sqlite3.connect('music.db') as connection:
                cursor = connection.cursor()

                sql_query = "SELECT * FROM music WHERE genre LIKE ? OR subgenre LIKE ?"

                cursor.execute(sql_query, ['%'+self.albumGenre+'%', '%'+self.albumGenre+'%'])
                results = cursor.fetchall()
                print(results)

                albums = []

                for result in results:
                    keys = ['index','year','title','artist','genre','subgenre','price']
                    vals = list(result)

                    result_dict = dict(zip(keys, vals))

                    album = Album(**result_dict)
                    albums.append(album.model_dump_json())

                return albums


        #print("Database connection closed successfully!")
        except sqlite3.IntegrityError as e:
            print(f"Error: Integrity constraint violated - {e}")

        except sqlite3.OperationalError as e:
            print(f"Error: Operational issue - {e}")

        except sqlite3.Error as e:
            print(f"Error: Generic Sqlite3 error - {e}")        

        except Exception as e:
            print(f"An unexpected error occurred: {e}")

In [8]:
class getAlbumByGenreAndYear(BaseModel):
    """Tool that queries albums by Genre and Year and returns a list of  Album objects containing the database index, Album Year, Album Title, Artist Name, Album Genre, Album Subgenre  and Album Price"""
    albumGenre: str = Field(description="The Album Genre name.")
    albumYear: str = Field(description="The Year.")

    def run(self):
        try:
            with sqlite3.connect('music.db') as connection:
                cursor = connection.cursor()

                sql_query = "SELECT * FROM music WHERE genre = ? and year=?"

                cursor.execute(sql_query, [self.albumGenre, self.albumYear])
                results = cursor.fetchall()
                print(results)

                albums = []

                for result in results:
                    keys = ['index','year','title','artist','genre','subgenre','price']
                    vals = list(result)

                    result_dict = dict(zip(keys, vals))

                    album = Album(**result_dict)
                    albums.append(album.model_dump_json())

                return albums


        #print("Database connection closed successfully!")
        except sqlite3.IntegrityError as e:
            print(f"Error: Integrity constraint violated - {e}")

        except sqlite3.OperationalError as e:
            print(f"Error: Operational issue - {e}")

        except sqlite3.Error as e:
            print(f"Error: Generic Sqlite3 error - {e}")        

        except Exception as e:
            print(f"An unexpected error occurred: {e}")

In [59]:
import csv
class writeToCsv(BaseModel):
    """Tool that writes to a csv file"""
    customerQuery: str = Field(description="The customer query.")
    queryAnswer: str = Field(description="The answer to the customer query.")

    def run(self):
        try:
            with open('email_output.csv','a', encoding='UTF8') as f:
                writer = csv.writer(f)
                data = [self.customerQuery, self.queryAnswer, 'yes']
                writer.writerow(data)
            print(f"PRINTING FROM CSV Writer:  {self.queryAnswer}")
            return self.queryAnswer
   

        except Exception as e:
            print(f"An unexpected error occurred: {e}")

In [60]:
# write function to execute tool. Take function name and parameters and return result of function
@traceable(name="AD Tool Call")
def execute_function(tool_call, tool_lookup):
    function_name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    tool = tool_lookup[function_name](**args)
    return function_name, tool.run()


In [64]:
@traceable(name="Albums Workflow")
def albumWorkflow(customerQuery: str):

    tools = [getAlbumByTitle, getAlbumByArtist, getAlbumByGenre, getAlbumByYear, getAlbumByGenreAndYear, writeToCsv]
    tool_lookup = {tool.__name__: tool for tool in tools}
    # message to read file
    messages = [
        {"role": "system", "content": "You are a useful music shop assistant that answers customer queries providing information on albums and writes the answer to a csv file"},
        {"role": "user", "content": customerQuery}

    ]

    response = client.chat.completions.create (
    model=model,
    messages=messages,
    temperature=0,
    tools=[pydantic_function_tool(tool) for tool in tools]
    )
    #print(f"Initial response: {response}")

    if not response.choices[0].message.tool_calls:
        # No tool calls needed, return direct response
        print("SURELEY NOT RETURNING HERE")
        return response
    
    tool_calls = response.choices[0].message.tool_calls
    #print(f"Tools to run: {tool_calls}")

    # Add the assistant's message with tool calls
    messages.append({
        "role": "assistant",
        "content": response.choices[0].message.content,
        "tool_calls": [
            {
                "id": tc.id,
                "type": "function",
                "function": {
                    "name": tc.function.name,
                    "arguments": tc.function.arguments
                }
            }
            for tc in tool_calls
        ]
    })

    # Execute tools
    for tool_call in tool_calls:
        function_name, tool_result = execute_function(tool_call, tool_lookup)
        
        # Add tool response to messages
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": str(tool_result)  
        })

    print(f"Messages after tool calls: {messages}")


    query_response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    print(f"QUERY _RESAPONSE:  {query_response.choices[0].message.content}")
    
    messages.append({
        "role": "assistant",
        "content": query_response.choices[0].message.content
    })
    print(f"Messages after assistant append: {messages}")

    messages.append(
        {
            "role" : "user",
            "content" : "write the customer query response to a csv file"
        }
    )



    
    write_response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        tools=[pydantic_function_tool(tool) for tool in tools]
    )

    if not write_response.choices[0].message.tool_calls:
        # No tool calls needed, return direct response
        print(f"WRITE RESPONSE: {write_response}")
        return write_response

    tool_calls = write_response.choices[0].message.tool_calls
    print(f"Tools to run: {tool_calls}")

    # Add the assistant's message with tool calls
    messages.append({
        "role": "assistant",
        "content": write_response.choices[0].message.content,
        "tool_calls": [
            {
                "id": tc.id,
                "type": "function",
                "function": {
                    "name": tc.function.name,
                    "arguments": tc.function.arguments
                }
            }
            for tc in tool_calls
        ]
    })

    # Execute tools
    for tool_call in tool_calls:
        function_name, tool_result = execute_function(tool_call, tool_lookup)
        
        # Add tool response to messages
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": str(tool_result)  
        })

    #print(f"Messages after tool calls: {messages}")
    print(f" THE QUERY ANSWER: {query_response.choices[0].message.content}")
    print(f"RETURN TYPE: {type(query_response.choices[0].message.content)}")
    return query_response.choices[0].message.content

In [62]:
res = albumWorkflow('do you have the album Nevermind ?')
res

Messages after tool calls: [{'role': 'system', 'content': 'You are a useful music shop assistant that answers customer queries providing information on albums and writes the answer to a csv file'}, {'role': 'user', 'content': 'do you have the album Nevermind ?'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'function-call-7626213736415145018', 'type': 'function', 'function': {'name': 'getAlbumByTitle', 'arguments': '{"albumTitle":"Nevermind"}'}}]}, {'role': 'tool', 'tool_call_id': 'function-call-7626213736415145018', 'content': '{"index":17,"year":1991,"title":"Nevermind","artist":"Nirvana","genre":"Rock","subgenre":"Alternative Rock, Grunge","price":0.0}'}]
QUERY _RESAPONSE:  Yes, we do have the album "Nevermind" by Nirvana. It was released in 1991 and is a classic of Alternative Rock and Grunge. The price is $0.0.

Would you like me to add it to your cart or provide more information about it?
Messages after assistant append: [{'role': 'system', 'content': 'You are a u

'Yes, we do have the album "Nevermind" by Nirvana. It was released in 1991 and is a classic of Alternative Rock and Grunge. The price is $0.0.\n\nWould you like me to add it to your cart or provide more information about it?'

In [41]:
res = albumWorkflow('do you have any bowie or led zep?')
res

Albums returned by Artist: ['{"index":35,"year":1972,"title":"The Rise and Fall of Ziggy Stardust and the Spiders From Mars","artist":"David Bowie","genre":"Rock","subgenre":"Classic Rock, Glam","price":0.0}']
Albums returned by Artist: ['{"index":35,"year":1972,"title":"The Rise and Fall of Ziggy Stardust and the Spiders From Mars","artist":"David Bowie","genre":"Rock","subgenre":"Classic Rock, Glam","price":0.0}', '{"index":108,"year":1971,"title":"Hunky Dory","artist":"David Bowie","genre":"Rock","subgenre":"Classic Rock, Glam","price":0.0}']
Albums returned by Artist: ['{"index":35,"year":1972,"title":"The Rise and Fall of Ziggy Stardust and the Spiders From Mars","artist":"David Bowie","genre":"Rock","subgenre":"Classic Rock, Glam","price":0.0}', '{"index":108,"year":1971,"title":"Hunky Dory","artist":"David Bowie","genre":"Rock","subgenre":"Classic Rock, Glam","price":0.0}', '{"index":251,"year":1977,"title":"Low","artist":"David Bowie","genre":"Electronic, Rock","subgenre":"Art 

"Yes, we have several albums by both David Bowie and Led Zeppelin in stock!\n\nFor **David Bowie**, we have:\n*   The Rise and Fall of Ziggy Stardust and the Spiders From Mars (1972)\n*   Hunky Dory (1971)\n*   Low (1977)\n*   Aladdin Sane (1973)\n*   Station to Station (1976)\n\nAnd for **Led Zeppelin**, we have:\n*   Led Zeppelin (1969)\n*   [Led Zeppelin IV] (1971)\n*   Physical Graffiti (1975)\n*   Led Zeppelin II (1969)\n*   Houses of the Holy (1973)\n\nI'll also save this information to a CSV file for you."

In [43]:
res = albumWorkflow('Do you have any David Bowie albums available? Cheers Tom')
res

Albums returned by Artist: ['{"index":35,"year":1972,"title":"The Rise and Fall of Ziggy Stardust and the Spiders From Mars","artist":"David Bowie","genre":"Rock","subgenre":"Classic Rock, Glam","price":0.0}']
Albums returned by Artist: ['{"index":35,"year":1972,"title":"The Rise and Fall of Ziggy Stardust and the Spiders From Mars","artist":"David Bowie","genre":"Rock","subgenre":"Classic Rock, Glam","price":0.0}', '{"index":108,"year":1971,"title":"Hunky Dory","artist":"David Bowie","genre":"Rock","subgenre":"Classic Rock, Glam","price":0.0}']
Albums returned by Artist: ['{"index":35,"year":1972,"title":"The Rise and Fall of Ziggy Stardust and the Spiders From Mars","artist":"David Bowie","genre":"Rock","subgenre":"Classic Rock, Glam","price":0.0}', '{"index":108,"year":1971,"title":"Hunky Dory","artist":"David Bowie","genre":"Rock","subgenre":"Classic Rock, Glam","price":0.0}', '{"index":251,"year":1977,"title":"Low","artist":"David Bowie","genre":"Electronic, Rock","subgenre":"Art 

In [44]:
res = albumWorkflow('What rock albums do you have from the early 70s?')
res

[(23, 1970, 'John Lennon/Plastic Ono Band', 'John Lennon / Plastic Ono Band', 'Rock', 'Pop Rock', 0.0), (51, 1970, 'Bridge Over Troubled Water', 'Simon & Garfunkel', 'Rock', 'Folk Rock, Classic Rock', 0.0), (74, 1970, 'After the Gold Rush', 'Neil Young', 'Rock', 'Rock & Roll, Country Rock', 0.0), (104, 1970, 'Sweet Baby James', 'James Taylor', 'Rock', 'Folk Rock, Acoustic, Soft Rock', 0.0), (110, 1970, 'Loaded', 'The Velvet Underground', 'Rock', 'Art Rock, Classic Rock', 0.0), (117, 1970, 'Layla and Other Assorted Love Songs', 'Derek and the Dominos', 'Rock', 'Blues Rock', 0.0), (131, 1970, 'Paranoid', 'Black Sabbath', 'Rock', 'Hard Rock, Heavy Metal', 0.0), (147, 1970, 'Deja vu', 'Crosby, Stills, Nash & Young', 'Rock', 'Classic Rock', 0.0), (170, 1970, 'Live at Leeds', 'The Who', 'Rock', 'Classic Rock, Blues Rock, Hard Rock', 0.0), (191, 1970, 'Fun House', 'The Stooges', 'Rock', 'Garage Rock, Punk', 0.0), (243, 1970, 'Black Sabbath', 'Black Sabbath', 'Rock', 'Blues Rock, Heavy Metal',

"Here are some rock albums from the early 70s that we have in stock:\n\n**1970:**\n*   John Lennon/Plastic Ono Band by John Lennon / Plastic Ono Band\n*   Bridge Over Troubled Water by Simon & Garfunkel\n*   After the Gold Rush by Neil Young\n*   Sweet Baby James by James Taylor\n*   Loaded by The Velvet Underground\n*   Layla and Other Assorted Love Songs by Derek and the Dominos\n*   Paranoid by Black Sabbath\n*   Deja vu by Crosby, Stills, Nash & Young\n*   Live at Leeds by The Who\n*   Fun House by The Stooges\n*   Black Sabbath by Black Sabbath\n*   American Beauty by Grateful Dead\n*   Workingman's Dead by Grateful Dead\n*   Let It Be by The Beatles\n*   All Things Must Pass by George Harrison\n*   Back in the USA by MC5\n\n**1971:**\n*   Who's Next by The Who\n*   Sticky Fingers by The Rolling Stones\n*   [Led Zeppelin IV] by Led Zeppelin\n*   Imagine by John Lennon\n*   Hunky Dory by David Bowie\n*   Electric Warrior by T. Rex\n*   Every Picture Tells a Story by Rod Stewart\n* 

In [45]:
res = albumWorkflow('What prog rock albums do you have?')
res

[(28, 1971, "Who's Next", 'The Who', 'Rock', 'Hard Rock, Mod, Prog Rock, Psychedelic Rock', 0.0), (43, 1973, 'The Dark Side of the Moon', 'Pink Floyd', 'Rock', 'Prog Rock', 0.0), (87, 1979, 'The Wall', 'Pink Floyd', 'Rock', 'Alternative Rock, Prog Rock', 0.0), (173, 1972, 'Something/Anything?', 'Todd Rundgren', 'Rock', 'Power Pop, Pop Rock, Prog Rock', 0.0), (211, 1975, 'Wish You Were Here', 'Pink Floyd', 'Rock', 'Classic Rock, Prog Rock', 0.0), (231, 1975, 'A Night at the Opera', 'Queen', 'Rock', 'Hard Rock, Pop Rock, Prog Rock', 0.0), (337, 1971, 'Aqualung', 'Jethro Tull', 'Rock', 'Classic Rock, Prog Rock', 0.0)]
Messages after tool calls: [{'role': 'system', 'content': 'You are a useful music shop assistant that answers customer queries providing information on albums and writes the answer to a csv file'}, {'role': 'user', 'content': 'What prog rock albums do you have?'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'function-call-8520978407612303807', 'type': 'funct

"Here are the progressive rock albums we have in stock:\n\n*   **Who's Next** by The Who (1971)\n*   **The Dark Side of the Moon** by Pink Floyd (1973)\n*   **The Wall** by Pink Floyd (1979)\n*   **Something/Anything?** by Todd Rundgren (1972)\n*   **Wish You Were Here** by Pink Floyd (1975)\n*   **A Night at the Opera** by Queen (1975)\n*   **Aqualung** by Jethro Tull (1971)\n\nI have also saved this information to a CSV file."

In [65]:
import csv

# Read CSV file and print each line
with open('emails.csv', 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)
    
    for line in csv_reader:
       print(line[0])
       res = albumWorkflow(line[0])  # Each line is a list of values
       res

Do you have any David Bowie albums available? Cheers Tom
Albums returned by Artist: ['{"index":35,"year":1972,"title":"The Rise and Fall of Ziggy Stardust and the Spiders From Mars","artist":"David Bowie","genre":"Rock","subgenre":"Classic Rock, Glam","price":0.0}']
Albums returned by Artist: ['{"index":35,"year":1972,"title":"The Rise and Fall of Ziggy Stardust and the Spiders From Mars","artist":"David Bowie","genre":"Rock","subgenre":"Classic Rock, Glam","price":0.0}', '{"index":108,"year":1971,"title":"Hunky Dory","artist":"David Bowie","genre":"Rock","subgenre":"Classic Rock, Glam","price":0.0}']
Albums returned by Artist: ['{"index":35,"year":1972,"title":"The Rise and Fall of Ziggy Stardust and the Spiders From Mars","artist":"David Bowie","genre":"Rock","subgenre":"Classic Rock, Glam","price":0.0}', '{"index":108,"year":1971,"title":"Hunky Dory","artist":"David Bowie","genre":"Rock","subgenre":"Classic Rock, Glam","price":0.0}', '{"index":251,"year":1977,"title":"Low","artist":